In [1]:
# !pip install torch==1.9
# !pip install torchtext==0.10.1
# Libraries
from konlpy.tag import Okt  
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter

import torch
import ast

# Preliminaries
from torchtext.legacy.data import TabularDataset ,Field, LabelField ,BucketIterator, Example, Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
# Models
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import ConcatDataset
from konlpy.tag import Okt  
# Training
import re
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

# Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [2]:
train_data = pd.read_excel("./middle/after_tokenized_train.xlsx")[["tokenized_text","label"]]
test_data = pd.read_excel("./middle/after_tokenized_test.xlsx")[["tokenized_text","label"]]

In [3]:
okt=Okt()  
with open('./middle/all_speech_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

all_morphs = okt.morphs(text)
counted_morphs = Counter(all_morphs)

In [4]:
tokens = list(counted_morphs.keys())
token_to_index = {token: i+1 for i, token in enumerate(tokens)}

In [5]:
def token_to_index_list(token_list):
    token_list = eval(token_list)
    ret = []
    for token in token_list:
        ret.append(token_to_index[token])
    return ret

In [6]:
train_data["indexed_token"] = train_data["tokenized_text"].map(token_to_index_list)
test_data["indexed_token"] = test_data["tokenized_text"].map(token_to_index_list)

In [7]:
train_data.head()

,tokenized_text,label,indexed_token
0,"['유소영', '비호감', '성형', '아줌마']",1,"[1, 2, 3, 4]"
1,"['나오지마라', '썅']",3,"[5, 6]"
2,"['식상하고', '지긋지긋했는데', '잘', '끝나네', '오예', '소리', '벗...",6,"[7, 8, 9, 10, 11, 12, 13, 14, 15]"
3,"['성희롱', '당할', '얼굴', '아닌데', 'ㅋㅋ']",5,"[16, 17, 18, 20, 21]"
4,"['끝', '까지', '해보자', '쪽바리', '원숭이', '자유', '한국', '...",0,"[22, 23, 24, 25, 26, 27, 28, 29, 30]"


In [7]:
indexed_tokens_list_test = [tokens for tokens in test_data['indexed_token']]
indexed_tokens_list_train = [tokens for tokens in train_data['indexed_token']]

In [14]:
import torch.nn.functional as F

def pad_sequences(sequences, max_len=None):
    if max_len is None:
        max_len = max([len(seq) for seq in sequences])
    padded_seqs = torch.zeros((len(sequences), max_len)).long()
    for i, seq in enumerate(sequences):
        end = min(len(seq), max_len)
        padded_seqs[i, :end] = torch.LongTensor(seq[:end])
    return padded_seqs

In [16]:
train_input_list = pad_sequences(indexed_tokens_list_train,121)
test_input_list = pad_sequences(indexed_tokens_list_test,121)
train_answer_list = [int(label) for label in train_data['label']]
train_answer_list = torch.tensor(train_answer_list)

In [21]:
torch.save(train_input_list, './processed_data/train_input.pt')
torch.save(test_input_list,  './processed_data/test_input.pt')
torch.save(train_answer_list, './processed_data/train_label.pt')